In [ ]:
#imports
import json 
import ast
import pandas as pd
import numpy as np
import time
import requests
import plotly
import plotly.express as px
print(plotly.__version__)
import ipykernel

: 

In [ ]:
# cache section so when i run code below that is doesn't take forever
refresh_cache = False
all_dasid_data = {}

: 

In [ ]:
#parameters for the algorithm

#nodes to get from the json files
nodes = 150 
#minimum rank of taxonomy to be included in the graph
min_rank = 'species'
#maximum rank of taxonomy to be included in the graph
max_rank = 'kingdom'
#% covered of the species in the original list of taxamonyms
covered_percentage = 1 #100% = 1 / 50% = 0.5
#qualified ranks of taxonomy to be included from aphia => default == accepted
qualified_ranks = 'accepted'

: 

In [ ]:
#read in the data from the csv file with pandas

df = pd.read_csv('aphia_ids_to_imis.csv')
#print the dimentions of the dataframe
print(df.shape)

(1017, 2)


: 

In [ ]:
# make child dataframes based on the parent dataframe 
children = {}
#get the different values that are present in the first column of the dataframe
diff_dasids = df['IMIS_DasID'].unique()

#loop over the different dasids and make a list of all the aphia_ids that are associated with that dasid
for dasid in diff_dasids:
    children[dasid] = df[df['IMIS_DasID'] == dasid]['aphia_id'].tolist()

#print the children dataframess
print(children)

{4221: [118852, 102101, 130926, 130537, 130357, 141595, 339374, 110398, 140480, 141579, 140380, 138998, 155177, 140126, 131187, 129876, 107552, 141424, 102317, 101742, 107381, 131177, 131495, 130359, 120020, 120206, 130491, 147474, 2036, 141134, 152438, 339379, 131170, 101361, 103073, 129379, 130272, 130404, 130616, 103068, 129884, 140430, 129868, 129370, 238384, 110487], 5609: [107380, 127160, 127141, 126977, 231748, 107552, 125999, 107032, 125546, 126281, 126426, 272027, 126975, 126942, 141444, 126425], 6451: [110483, 152249, 146950, 506128, 140032, 130623, 125579, 152217, 101669, 100982, 131135, 111659, 1609, 111367, 712167, 129781, 799, 140627, 534781, 101981, 131321, 985, 140103, 130491, 931, 130523, 150520, 123776, 102720, 101489, 134599, 10194, 101930, 138223, 129710, 757970, 102036, 130131, 130738, 119898, 131170, 131193, 102611, 834002, 130128, 836020, 102408, 506025, 445590, 196385, 1564, 130616, 130722, 327985, 130407, 152316, 1130, 131074, 110467, 141433, 107742, 196003, 13

: 

In [ ]:
#make requests to the aphia api to get the taxonomy of the aphia_ids
if refresh_cache:
    for dasid, aphia_ids in children.items():
        print(f"beginning search on dasid: {dasid}")
        length_aphia_ids = len(aphia_ids)
        i = 1
        covered_percentage = 0
        for aphia_id in aphia_ids:
            #get percentge of the aphia_ids that are covered (i/length_aphia_ids)*100, if the percentage is higher than the covered_percentage, then print message
            percentage_covered = (i/length_aphia_ids)*100
            i=i+1
            if percentage_covered > covered_percentage:
                print(f"{dasid} | {percentage_covered}'%' done | {i}/{length_aphia_ids}")
                covered_percentage = covered_percentage + 10
            if i % 50 == 0:
                print(f"{dasid} | {percentage_covered}'%' done | {i}/{length_aphia_ids}")
            #check if the aphia_id is in the cache
            try:
                if aphia_id in all_dasid_data[dasid]:
                    pass
                else:
                    url_to_request = 'https://www.marinespecies.org/rest/AphiaClassificationByAphiaID/' + str(aphia_id)
                    aphia_id_data = requests.get(url_to_request).json()
                    all_dasid_data[dasid][aphia_id] = aphia_id_data
                    time.sleep(1.5)
            except:
                all_dasid_data[dasid] = {}
                url_to_request = 'https://www.marinespecies.org/rest/AphiaClassificationByAphiaID/' + str(aphia_id)
                aphia_id_data = requests.get(url_to_request).json()
                all_dasid_data[dasid][aphia_id] = aphia_id_data
                time.sleep(1.5)
        
        #write the data to a json file
        # Directly from dictionary
        utf_8_encoded_dict_encoded= {str(k).encode("utf-8"): str(v).encode("utf-8") for k,v in all_dasid_data.items()}
        utf_8_encoded_dict = {k.decode("utf-8"): v.decode("utf-8") for k,v in utf_8_encoded_dict_encoded.items()}
        new_dict = {}
        for key, value in utf_8_encoded_dict.items():
            new_dict[key] = ast.literal_eval(value.replace("'", "\""))
        with open('json_data_aphia_ids.json', 'w') as outfile:
            json.dump(new_dict, outfile)


: 

In [ ]:
#read in the data from the json file
f = open('json_data_aphia_ids.json', 'r')
data = json.load(f)

dasids = list(data.keys())
print(dasids)

def check_dict_aphia_id(checkvalue, array_aphia_ids, parent_aphia_id, character, value_dict, parent_aphia_ids):
    #check if checkvalue is of type dict
    if isinstance(checkvalue, dict):
        #check if there is a key named 'AphiaID'
        for key, value in checkvalue.items():
            if key == 'AphiaID':
                array_aphia_ids.append(value)
                #check if the value is already in charcter array and if not, add it and append 1 to value array | if yes find the index of the value in the array and add 1 to the value array
                if value in character:
                    index = character.index(value)
                    value_dict[index] = value_dict[index] + 1
                else:
                    character.append(value)
                    value_dict.append(1)
                    parent_aphia_ids.append(parent_aphia_id)
            if key == 'child':
                check_dict_aphia_id(value, array_aphia_ids, parent_aphia_id=checkvalue["AphiaID"], character= character, value_dict = value_dict, parent_aphia_ids = parent_aphia_ids)
    return array_aphia_ids

#get all aphia_ids from each dasid
dasids_unique_aphia_ids = {}
sunburst_chart_data_dasids = {}

for dasid in dasids:
    all_aphia_ids = []
    parent_aphia_ids = []
    character = []
    value_dict = []
    data_dasid = data[dasid]
    for aphia_id , value_aphia_id in data_dasid.items():
        all_aphia_ids.append(aphia_id)
        toappend_later_aphia_ids = []
        check_dict_aphia_id(value_aphia_id, toappend_later_aphia_ids, "", character=character, value_dict=value_dict, parent_aphia_ids=parent_aphia_ids)
        for id_aphia in toappend_later_aphia_ids:
            all_aphia_ids.append(id_aphia)
    #get unique values of all_aphia_ids
    unique_aphia_ids = list(set(all_aphia_ids))
    print(f" dasid: {dasid} | unique aphia_ids: {len(unique_aphia_ids)}")
    dasids_unique_aphia_ids[dasid] = unique_aphia_ids
    sunburst_chart_data_dasids[dasid] = {'character': character, 'value': value_dict, 'parent_aphia_ids': parent_aphia_ids}
        #check if value_aphia_id as child property that is of type dict => if true then check in the keys of the dict if there is an aphia id and then add it + check if there is a child in the dict

['4221', '5609', '6451', '6575', '7890', '7971', '7972']
 dasid: 4221 | unique aphia_ids: 253
 dasid: 5609 | unique aphia_ids: 109
 dasid: 6451 | unique aphia_ids: 2290
 dasid: 6575 | unique aphia_ids: 26
 dasid: 7890 | unique aphia_ids: 858
 dasid: 7971 | unique aphia_ids: 16
 dasid: 7972 | unique aphia_ids: 53


: 

In [ ]:
# show figures of sunburst chart for each dasid

#print the current version of plotly



for dasid, sb_data in sunburst_chart_data_dasids.items():
    fig = px.sunburst(
        sb_data,
        names='character',
        parents='parent',
        values='value',
    )
    fig.show()

AttributeError: module 'plotly.express' has no attribute 'sunburst'

: 